**Autor: Mariana Chaves**
 
**Date: July 2022**

This notebooks reproduces the example presented by the authors of ProtoryNet in their paper and their [github repository](https://github.com/dathong/ProtoryNet), where they classify positive and negative reviews. 
We show that after a couple of epochs the model achieves 93.56% of accuracy. 
Nevertheless, the trained prototypes and their scores suggest that all prototypes are linked to the category "positive" almost to the same degree.
With such prototypes one would expect the the model would classify all samples as "positive".
Hence, even correct classifications seem unrelated to the prototypes.
It seems that, after mapping each sentence to its active prototype, the model is still taking some decisions which remain occluded in the layers that follow the prototype layer.

This notebook is largely inspired by the example presented by the authors of ProtoryNet in their [github repository](https://github.com/dathong/ProtoryNet).

# Importing packages

In [1]:
#import neccesary packages
import tensorflow_hub as hub
import tensorflow as tf
import pickle
from keras import backend as K
import numpy as np
from sklearn_extra.cluster import KMedoids
from tensorflow import keras
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from datetime import datetime
from scipy.spatial import distance_matrix
import sys
import time
sys.path.append('../src/protoryNet/')
from protoryNet import ProtoryNet

In [ ]:
# Go to main directory
%cd ..

# Import datasets (original example of the hotel)

We use the same dataset as the authors of ProtoryNet.

In [6]:
#directory of the datasets. Github provides these files in the form of pickle files
#dir = "../datasets/hotel/"
# dir = "/content/drive/Othercomputers/My Laptop/Documents/UCA DSAI/Semester 3/Research project/ProtoryNet/datasets/hotel/"
dir = "/nfshome/students/cm007951/protorynet/datasets/hotel/"
with open (dir + 'y_train', 'rb') as fp:
    y_train = pickle.load(fp)

with open (dir + 'train_not_clean', 'rb') as fp:
    train_not_clean = pickle.load(fp)

with open (dir + 'test_not_clean', 'rb') as fp:
    test_not_clean = pickle.load(fp)

with open (dir + 'y_test', 'rb') as fp:
    y_test = pickle.load(fp)

In [7]:
print(train_not_clean[:2])
print(y_train[:2])
len(train_not_clean)

["RENT A STEAM CLEANER. The location is very, very inconvenient, but I am sure that is nothing they can help, it is just how the city has grown around their location. I would have rated the room as a 4 or 5 for clean if the sofa was not so disgusting. I literally covered it with a sheet just to sit on it. There was not one inch of surface that was not covered with a greasy/dirty stain. I would have switched rooms if I hadn't already unloaded all of my stuff in the rain before I turned on the lights to reveal the nasty sofa. Otherwise, the room was clean and comfortable. It would be worth a couple of hundred dollars to get a steam cleaner and clean the sofas in each room. Not to mention how unsanitary it is", "Great hotel and staff - recommend. I came in late after a long drive from Vegas ending in a whole lot of highway exits and concentration to get to Napa. I was greeted by a friendly night staff who was expecting me, and (not sure of his name) made me feel more relaxed immediately. 

3606

# Data preprocessing

In [8]:
#this method is to split the paragraphs into sentences
def gen_sents(para):
    res = []
    for p in para:
        sents = p.split(".")
        res.append(sents)
    return res


train_noclean_sents = gen_sents(train_not_clean)
test_noclean_sents = gen_sents(test_not_clean)

In [9]:
train_noclean_sents[:2]

[['RENT A STEAM CLEANER',
  ' The location is very, very inconvenient, but I am sure that is nothing they can help, it is just how the city has grown around their location',
  ' I would have rated the room as a 4 or 5 for clean if the sofa was not so disgusting',
  ' I literally covered it with a sheet just to sit on it',
  ' There was not one inch of surface that was not covered with a greasy/dirty stain',
  " I would have switched rooms if I hadn't already unloaded all of my stuff in the rain before I turned on the lights to reveal the nasty sofa",
  ' Otherwise, the room was clean and comfortable',
  ' It would be worth a couple of hundred dollars to get a steam cleaner and clean the sofas in each room',
  ' Not to mention how unsanitary it is'],
 ['Great hotel and staff - recommend',
  ' I came in late after a long drive from Vegas ending in a whole lot of highway exits and concentration to get to Napa',
  ' I was greeted by a friendly night staff who was expecting me, and (not sur

In [10]:
x_train = train_noclean_sents
x_test = test_noclean_sents

#Make sure the label values are integers
y_train = [int(y) for y in y_train]
y_test = [int(y) for y in y_test]


print('test_noclean_sents ', len(test_noclean_sents), len(test_noclean_sents[0]), test_noclean_sents[0])
print('y_train: ', y_train[:10])
print('y_test: ', y_test[:10])

test_noclean_sents  902 7 ['Share your room with roach or bug', ' Roach in the room but the staff said its just a bug not a roach, like that made it any better', ' Big hole in the bathroom wall were this bug/roach came out of', ' Wonder what other rodent will come out of the wall', 'Thank you for providing us with your feedback', ' I would like to apologize for any of the inconveniences you may have experienced during your time with us', ' I wish that I had been notified of your concerns before you had checked out and I would like to apologize on behalf of our staff']
y_train:  [1, 1, 1, 0, 1, 1, 0, 0, 1, 1]
y_test:  [0, 0, 1, 1, 0, 1, 0, 0, 0, 1]


In [11]:
#import Google Sentence encoder, to convert sentences into vector values
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("module %s loaded" % module_url)

def embed(input):
    return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


Take the first 10000 reviews of the training set and embed them using Google Sentence Encoder.
We cannot take all reviews because later we use k-medoids, which runs into memery issues if it has too many data points.
Each review can contain more than one sentence, that's why the 1st dimention of the object is 41996. 
512 corresponds to the size of the latent space.  

In [13]:
sample_sentences = []
for p in train_noclean_sents[:10000]:
    sample_sentences.extend(p)

#compute vector values of sentences
sample_sent_vect = embed(sample_sentences)
# print(sample_sent_vect)
print(sample_sent_vect.shape)

(41996, 512)


In [13]:
# We put each sentence separately, that's why the number increases
print(len(train_noclean_sents))
print(len(sample_sentences))
print(train_noclean_sents[0])
print(sample_sentences[0:10])

3606
11747
['RENT A STEAM CLEANER', ' The location is very, very inconvenient, but I am sure that is nothing they can help, it is just how the city has grown around their location', ' I would have rated the room as a 4 or 5 for clean if the sofa was not so disgusting', ' I literally covered it with a sheet just to sit on it', ' There was not one inch of surface that was not covered with a greasy/dirty stain', " I would have switched rooms if I hadn't already unloaded all of my stuff in the rain before I turned on the lights to reveal the nasty sofa", ' Otherwise, the room was clean and comfortable', ' It would be worth a couple of hundred dollars to get a steam cleaner and clean the sofas in each room', ' Not to mention how unsanitary it is']
['RENT A STEAM CLEANER', ' The location is very, very inconvenient, but I am sure that is nothing they can help, it is just how the city has grown around their location', ' I would have rated the room as a 4 or 5 for clean if the sofa was not so d

# Prototype initialization

The initial prototypes are centroids chosen by k-medoids.  

In [14]:
k_protos, vect_size = 10, 512 #512 because we have the sentences are transformed into vectors of size 512
kmedoids = KMedoids(n_clusters=k_protos, random_state=0).fit(sample_sent_vect)
k_cents = kmedoids.cluster_centers_
print(k_cents.shape)

(10, 512)


In [15]:
print(k_cents)

[[-3.0867750e-02 -4.0647369e-02 -6.2585823e-02 ... -4.3438803e-02
   2.1392105e-02 -1.9520901e-02]
 [ 2.4763636e-02 -7.5006582e-02 -5.5820044e-02 ... -1.8545931e-02
   6.3773416e-02  2.4646815e-02]
 [-2.7673064e-02 -1.2406837e-03 -4.8067428e-02 ... -8.5736131e-03
   4.5512866e-02  1.8920101e-02]
 ...
 [-3.9867338e-02  1.2999296e-02 -4.1530743e-02 ... -2.1137370e-02
  -2.4923310e-04  5.6575678e-02]
 [-3.8432319e-02 -2.3378124e-03 -2.9857373e-02 ... -5.9871352e-05
   9.3568325e-02  2.2465114e-02]
 [-1.7571883e-02 -4.6657540e-02 -2.1202940e-02 ... -4.3001492e-04
   7.2558373e-02  3.1543452e-02]]


# Model training

In [16]:
pNet = ProtoryNet() 

In [17]:
model = pNet.createModel(k_cents)

[db] model.input =  KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name='input_1'), name='input_1', description="created by layer 'input_1'")
[db] protoLayerName =  proto_layer
[db] protoLayer =  <protoryNet.ProtoryNet.createModel.<locals>.prototypeLayer object at 0x7f9a512e4490>
[db] protoLayer.output =  (<KerasTensor: shape=(1, None, 10) dtype=float32 (created by layer 'proto_layer')>, <KerasTensor: shape=(10, 512) dtype=float32 (created by layer 'proto_layer')>)
[db] distanceLayer.output =  KerasTensor(type_spec=TensorSpec(shape=(1, None, 10), dtype=tf.float32, name=None), name='distance_layer/PartitionedCall:0', description="created by layer 'distance_layer'")
Model: "custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None,)]                 0         
                                                                 
 keras_layer (KerasLaye

In [18]:
start_time = time.time()
pNet.train(x_train,y_train,x_test,y_test, epochs=1, saveModel=True, model_name="test_hotel")
execution_time = (time.time() - start_time) / 60
print(execution_time)

Epoch  0
i =   0
1/1 [==============================] - 11s 11s/step - loss: 0.2301
Evaluate on valid set:  0.5144124168514412
This is the best eval res, saving the model...
saving model now = 2022-07-21 11:12:10.517628
just saved
i =   50
1/1 [==============================] - 0s 79ms/step - loss: 0.2405
i =   100
1/1 [==============================] - 0s 79ms/step - loss: 1.3593
i =   150
1/1 [==============================] - 0s 85ms/step - loss: 0.4773
i =   200
1/1 [==============================] - 0s 77ms/step - loss: 1.1163
Evaluate on valid set:  0.5144124168514412
i =   250
1/1 [==============================] - 0s 85ms/step - loss: 0.9968
i =   300
1/1 [==============================] - 0s 78ms/step - loss: 0.4902
i =   350
1/1 [==============================] - 0s 85ms/step - loss: 0.5020
i =   400
1/1 [==============================] - 0s 84ms/step - loss: 1.1189
Evaluate on valid set:  0.5144124168514412
i =   450
1/1 [==============================] - 0s 78ms/step - loss

# Model testing (from saved model)

We want to work with the model when the validation accuracy was the best.
Therefore, we load the saved model.

In [14]:
nprototypes = 10
model_path = 'test_hotel.h5'

pNet_saved = ProtoryNet()
model = pNet_saved.createModel(np.zeros((nprototypes, 512)), nprototypes)
model.load_weights(model_path)

[db] model.input =  KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name='input_1'), name='input_1', description="created by layer 'input_1'")
[db] protoLayerName =  proto_layer
[db] protoLayer =  <protoryNet.ProtoryNet.createModel.<locals>.prototypeLayer object at 0x7f7adc1e4a10>
[db] protoLayer.output =  (<KerasTensor: shape=(1, None, 10) dtype=float32 (created by layer 'proto_layer')>, <KerasTensor: shape=(10, 512) dtype=float32 (created by layer 'proto_layer')>)
[db] distanceLayer.output =  KerasTensor(type_spec=TensorSpec(shape=(1, None, 10), dtype=tf.float32, name=None), name='distance_layer/PartitionedCall:0', description="created by layer 'distance_layer'")
Model: "custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None,)]                 0         
                                                                 
 keras_layer (KerasLaye

Since the embedder is trained along which the model, we cannot use anymore the embeddings from Google Universal Encoder. 
We need to use the fine-tuned embedder of the model. 
We can access it through ```.embed()```.
We embed the sentences using the new embedder.
We need these embedded sentences to later see the prototypes of the model. 

In [21]:
sample_sentences_embedded = pNet_saved.embed(sample_sentences)

The test accuracy is approximately 93%

In [15]:
pNet_saved.evaluate(x_test, y_test)

([0.009236685,
  0.8359412,
  0.98266166,
  0.9742343,
  0.030007662,
  0.97888786,
  0.26315978,
  0.085663304,
  0.009625662,
  0.97489357,
  0.9887173,
  0.98883736,
  0.16210902,
  0.9827943,
  0.010352292,
  0.97174305,
  0.9790254,
  0.97521985,
  0.57795393,
  0.9856171,
  0.9535169,
  0.99347126,
  0.26618475,
  0.63590795,
  0.6674057,
  0.9797632,
  0.07994812,
  0.9860044,
  0.9936752,
  0.029118944,
  0.99273694,
  0.03949093,
  0.96651024,
  0.042999916,
  0.9696063,
  0.9711973,
  0.89880747,
  0.0654446,
  0.45315576,
  0.9792458,
  0.2899294,
  0.9675286,
  0.06526422,
  0.966731,
  0.9899165,
  0.009109211,
  0.0312802,
  0.98854095,
  0.2522606,
  0.069065206,
  0.48740974,
  0.9804944,
  0.9894852,
  0.9170779,
  0.6747042,
  0.41735515,
  0.04179582,
  0.9721919,
  0.9882619,
  0.94456625,
  0.009794364,
  0.012702928,
  0.22117573,
  0.6056591,
  0.9825514,
  0.9683224,
  0.07020599,
  0.57513773,
  0.06275198,
  0.9929115,
  0.576648,
  0.78463197,
  0.033086147,


Visualize the prototypes. 

In [22]:
prototypes = pNet_saved.showPrototypes(sample_sentences, sample_sentences_embedded, nprototypes, printOutput=False, return_prototypes = True)
prototypes

{0: "I'd stay here again for sure",
 1: 'Never was able to get a shuttle the entire time we were there',
 2: "It was my first stay in a kimpton hotel and now I don't want to stay anywhere else",
 3: 'Was a small hotel which was fine for us',
 4: "I would have given this hotel a 5 star but we didn't like the parking situation",
 5: 'I am happy to hear that you enjoyed our great location',
 6: 'We stop here often if we cannot get a decent price in a WDW resort',
 7: 'And if you need pharmacy or coffee fix, they are all along the Main St right outside the hotel',
 8: 'More',
 9: 'We chose Hotel Abri because it was over 100 less than a place across the street'}

Below, the model correctly classifies a positive review.
The prediction is 0.66, since is it above 0.5, the prediction is "positive review".

In [30]:
testS = ["Generally I like the hotel and will come back",
         "It's a small hotel",
         "The room is nice, clean and suitable for a nice stay however",
         "It's one of the best in the area"]
pNet_saved.predict(testS)

array([0.6664621], dtype=float32)

Now we see the prototype associated to each sentence.

In [31]:
trajectory = pNet_saved.showTrajectory(testS, sample_sentences, sample_sentences_embedded)
print(trajectory)

['I am happy to hear that you enjoyed our great location', 'Was a small hotel which was fine for us', 'I am happy to hear that you enjoyed our great location', 'Was a small hotel which was fine for us']


In [32]:
def score_trajectory(list_of_sentences):
    '''
    given a list of sentences (usually a list of prototypes), it returns the prediction for each of them
    '''
    pred = []
    for prot in list_of_sentences:
        pred.append(pNet_saved.predict([prot])[0])
    return pred

The scores associated to each prototype are shown below.  

**Note that all prototypes by their own would give a positive score, that is, the prediction for each prototype would be "positive review", nevertheless the model classifies correctly negative reviews.**
If the model was trully only mapping the sentences to the prototypes then taking decisions based on them the predictions would be always "positive review". 
Therefore, after mapping each sentence to the prototypes, the model is still taking some decisions, which remain occluded in the layers after the prototype layer. 
This indicates that protorynet is not as transparent as their authors claim it to be. 

In [33]:
score_trajectory(prototypes.values())

[0.7718854,
 0.70234185,
 0.70097613,
 0.77185166,
 0.77171296,
 0.77266157,
 0.7023217,
 0.77139384,
 0.69554263,
 0.7715117]

In [34]:
score_trajectory(trajectory)

[0.77266157, 0.77185166, 0.77266157, 0.77185166]

# Examples where the model fails at explainability

See how the model correctly classifies a negative review:

In [35]:
testS = x_test[0]
print(testS)
pNet_saved.predict(testS)

['Share your room with roach or bug', ' Roach in the room but the staff said its just a bug not a roach, like that made it any better', ' Big hole in the bathroom wall were this bug/roach came out of', ' Wonder what other rodent will come out of the wall', 'Thank you for providing us with your feedback', ' I would like to apologize for any of the inconveniences you may have experienced during your time with us', ' I wish that I had been notified of your concerns before you had checked out and I would like to apologize on behalf of our staff']


array([0.00923669], dtype=float32)

Nevertheless, the prototypes associated to each sentences give no information about why it was classified as negative. 
Moreover, all prototypes are the same. 

In [36]:
# Prototypes related to each sentence
trajectory = pNet_saved.showTrajectory(testS, sample_sentences, sample_sentences_embedded)
print(trajectory)

['More', 'More', 'More', 'More', 'More', 'More', 'More']


In this other example, the model correctly classifies a negative review, but one more the prototypes give little explanaitions. 

In [49]:
testS = x_test[7]
print(testS)
pNet_saved.predict(testS)

['Worst ever ', ' Room was nasty falling apart poorly kept up it ended up being the worst stay ever if it wasnt the only thing available for a big concert weekend we would of canceled ', 'the door had electrical tape covering up around the door knob , the locks did not work, we only had one light that worked,the microwave was covered in', ' More']


array([0.0856633], dtype=float32)

In [50]:
trajectory = pNet_saved.showTrajectory(testS, sample_sentences, sample_sentences_embedded)
print(trajectory)

['More', 'More', 'More', 'More']
